In [1]:
import traceback
import sys
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests

In [2]:
# Websites to scrape
nasa_mars_news_url = "https://mars.nasa.gov/news/"
jpl_mars_space_images_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
mars_facts_url = "https://space-facts.com/mars/"
astrogeology_url = "https://astrogeology.usgs.gov"
mars_hemispheres_url = astrogeology_url + "/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [3]:
class ChromeBrowser(object): 
    """
    """
    def __init__(self, url):
        self.url = url 
      
    def __enter__(self): 
        executable_path = {'executable_path': ChromeDriverManager().install()}
        self.browser = Browser('chrome', **executable_path, headless=False)
        self.browser.visit(self.url)
        return self.browser
  
    def __exit__(self, exc_type, exc_val, exc_tb):
        if (exc_type is not None):
            traceback.print_exception(exc_type, exc_val, exc_tb)
        if (self.browser is not None):
            self.browser.quit() 
        return True

In [4]:
# Scrape the NASA Mars News Site and collect the latest News Title 
# and Paragraph Text. Assign the 
# text to variables that you can 
# reference later.
news_title = ""
news_p = ""

with ChromeBrowser(nasa_mars_news_url) as browser: 
    html = browser.html
    soup = bs(html, "html.parser")
    news_title = soup.find('div', class_='content_title').text
    news_p = soup.find('div', class_='article_teaser_body').text
    
print(news_title)
print()
print(news_p)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Bootcamp\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache




Mars Now

Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  


In [11]:
# Use splinter to navigate the site and find the image url 
# for the current Featured Mars Image and assign the url 
# string to a variable called `featured_image_url`.

# Make sure to find the image url to the full size `.jpg` image.

# Make sure to save a complete url string for this image.

featured_image_url = ""

with ChromeBrowser(jpl_mars_space_images_url) as browser: 
    html = browser.html
    soup = bs(html, "html.parser")
    browser.links.find_by_partial_text("FULL IMAGE").click()
    html = browser.html
    soup = bs(html, "html.parser")
    featured_image_url = jpl_mars_space_images_url.\
    replace(
        "index.html", 
        soup.find(class_="fancybox-image")["src"]
    ) 
    
print(featured_image_url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Bootcamp\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache




https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg


In [6]:
# Visit the Mars Facts webpage and use Pandas to scrape the table
# containing facts about the planet including Diameter, Mass, etc.

# Use Pandas to convert the data to a HTML table string.

mars_facts = pd.read_html(mars_facts_url, match="Mars")[0]
mars_facts = mars_facts.set_index("Mars - Earth Comparison", drop=True)
mars_facts = mars_facts.transpose()
mars_facts.columns = [col.replace(":","") for col in mars_facts.columns]
#mars_facts["Diameter"] = mars_facts["Diameter"].replace({" km":"",",":"",}, regex=True).astype('float')
#mars_facts["Distance from Sun"] = mars_facts["Distance from Sun"].replace({" km":"",",":"",}, regex=True).astype('float')
mars_facts

,Diameter,Mass,Moons,Distance from Sun,Length of Year,Temperature
Mars,"6,779 km",6.39 × 10^23 kg,2,"227,943,824 km",687 Earth days,-87 to -5 °C
Earth,"12,742 km",5.97 × 10^24 kg,1,"149,598,262 km",365.24 days,-88 to 58°C


In [9]:
# Visit the USGS Astrogeology site to obtain high resolution images
# for each of Mar's hemispheres.

# You will need to click each of the links to the hemispheres in 
# order to find the image url to the full resolution image.

# Save both the image url string for the full resolution hemisphere 
# image, and the Hemisphere title containing the hemisphere name. 
# Use a Python dictionary to store the data using the keys `img_url`
# and `title`.

# Append the dictionary with the image url string and the hemisphere 
# title to a list. This list will contain one dictionary for each
# hemisphere.

hemisphere_image_urls = []

with ChromeBrowser(mars_hemispheres_url) as browser: 
    html = browser.html
    soup = bs(html, "html.parser")
    results = soup.find(class_="result-list").find_all(class_="item")
    for result in results:
        hemisphere = {}
        hemisphere["title"] = result.find("h3").text
        browser.links.find_by_partial_text(hemisphere["title"]).click()
        html = browser.html
        soup = bs(html, "html.parser")
        hemisphere["img_url"] = astrogeology_url + soup.find(class_="wide-image")["src"]
        hemisphere_image_urls.append(hemisphere)
        browser.back()
        
for url in hemisphere_image_urls:
    print(url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Bootcamp\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache




{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}
{'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}
{'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}
{'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}
